Code from:  
https://medium.com/ai-society/gans-from-scratch-1-a-deep-introduction-with-code-in-pytorch-and-tensorflow-cb03cdcdba0f

In [0]:
%mkdir dataset

In [1]:
!pip install tensorboardx

     |████████████████████████████████| 204kB 4.7MB/s 


In [0]:
import torch
from torch import nn, optim
from torch.autograd.variable import Variable
from torchvision import transforms, datasets

In [0]:
def mnist_data():
  compose = transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Normalize((.5,.5,.5), (.5,.5,.5))
  ])
  out_dir = './dataset'
  return datasets.MNIST(root=out_dir, train=True, transform=compose, download=True)

In [10]:
data = mnist_data()
data_loader = torch.utils.data.DataLoader(data, batch_size=100, shuffle=True)
num_batches = len(data_loader)

HTTPError: ignored

In [0]:
class DiscriminatorNet(torch.nn.Module):
  def __init__(self):
    super(DiscriminatorNet, self).__init__()
    n_features = 784
    n_out = 1

    self.hidden0 = nn.Sequential(
        nn.Linear(n_features, 1024),
        nn.LeakyReLU(0.2),
        nn.Dropout(0.3)
    )

    self.hidden1 = nn.Sequential(
        nn.Linear(1024, 512),
        nn.LeakyReLU(0.2),
        nn.Dropout(0.3)
    )

    self.hidden2 = nn.Sequential(
        nn.Linear(512, 256),
        nn.LeakyReLU(0.2),
        nn.Dropout(0.3)
    )

    self.out = nn.Sequential(
        nn.Linear(256, n_out),
        nn.Sigmoid()
    )

  def forward(self, x):
    x = self.hidden0(x)
    x = self.hidden1(x)
    x = self.hidden2(x)
    x = self.out(x)
    return x

discriminator = DiscriminatorNet()

In [0]:
def images_to_vectors(images):
  # images.size(0): Batch size
  return images.view(images.size(0), 784)

def vectors_to_images(vectors):
# vectors.size(0): Batch size
  return vectors.view(vectors.size(0),1,28,28)

In [0]:
class GeneratorNet(nn.Module):
  def __init__(self):
    super(GeneratorNet, self).__init__()
    n_features = 100
    n_out = 784

    self.hidden0 = nn.Sequential(
        nn.Linear(n_features, 256),
        nn.LeakyReLU(0.2)
    )

    self.hidden1 = nn.Sequential(
        nn.Linear(256, 512),
        nn.LeakyReLU(0.2)
    )

    self.hidden2 = nn.Sequential(
        nn.Linear(512, 1024),
        nn.LeakyReLU(0.2)
    )

    self.out = nn.Sequential(
        nn.Linear(1024, n_out),
        nn.Tanh()
    )

  def forward(self, x):
    x = self.hidden0(x)
    x = self.hidden1(x)
    x = self.hidden2(x)
    x = self.out(x)
    return x

generator = GeneratorNet()

In [0]:
def noize(size):
  n = Variable(torch.randn(size, 100))
  return n

In [0]:
d_optimizer = optim.Adam(discriminator.parameters(),lr = 0.0002)
g_optimizer = optim.Adam(generator.parameters(), lr=0.0002)

In [0]:
loss = nn.BCELoss()

In [0]:
def ones_target(size):
  data = Variable(torch.ones(size, 1))
  return data

def zeros_target(size):
  data = Variable(torch.zeros(size, 1))
  return data


In [0]:
def train_discriminator(optimizer, real_data, fake_data):
  N = real_data.size(0)
  optimizer.zero_grad()

  # 1.1 Train on Real Data
  prediction_real = discriminator(real_data)
  error_real = loss(prediction_real, ones_target(N))
  error_real.backward()

  # 1.2 Train on Fake Data
  prediction_fake = discriminator(fake_data)
  error_fake = loss(prediction_fake, zeros_target(N))
  error_fake.backward()

  # 1.3 Update weights with gradients
  optimizer.step()

  return error_real + error_fake, prediction_real, prediction_fake
